<a href="https://colab.research.google.com/github/cstorm125/align_use/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# #get stuff ready for colab
# !pip install tensorflow_text
# !wget https://www.dropbox.com/s/nr5h3xuqhlvj94m/europarl-v7.fr-en.en
# !wget https://www.dropbox.com/s/z730widwr5kw0zm/europarl-v7.fr-en.fr
!ls

europarl-v7.fr-en.en  europarl-v7.fr-en.fr  sample_data


In [0]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf #tensorflow 2.1.0

_model = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

In [7]:
#read files
with open('europarl-v7.fr-en.fr','r') as f: fr = f.readlines()
with open('europarl-v7.fr-en.en','r') as f: en = f.readlines()
len(fr), len(en)

(2007723, 2007723)

In [9]:
#create dataframe
df = pd.DataFrame({'fr':[i[:-1] for i in fr],'en':[i[:-1] for i in en]})
df.tail()

,fr,en
2007718,"Je me permettrai même, bien qu'ils soient abse...","I would also like, although they are absent, t..."
2007719,Je ne rouvrirai pas le débat sur le millénaire...,I am not going to re-open the 'Millennium or n...
2007720,Interruption de la session,Adjournment of the session
2007721,Je déclare interrompue la session du Parlement...,I declare the session of the European Parliame...
2007722,(La séance est levée à 10h50),(The sitting was closed at 10.50 a.m.)


In [11]:
%time x = _model(df['fr'][:10000])

CPU times: user 1 s, sys: 228 ms, total: 1.23 s
Wall time: 1.11 s


In [27]:
x = tf.random.normal([3000,512])@tf.random.normal([512,3000])
# tf.cast(x >0,tf.int32).numpy()
x

<tf.Tensor: shape=(3000, 3000), dtype=float32, numpy=
array([[-21.836205  ,  15.390707  ,  12.076106  , ..., -18.791527  ,
          0.8299967 ,  28.336113  ],
       [  0.85433674,   8.562556  , -35.347317  , ..., -12.781753  ,
          1.3013941 ,   1.4134177 ],
       [ -3.0149612 , -44.33893   , -56.680443  , ...,  -5.2696595 ,
         -5.9382973 , -23.926512  ],
       ...,
       [ 51.981487  ,   0.29452467,  33.96473   , ...,  10.3710785 ,
        -40.499256  ,  14.382589  ],
       [-20.679998  ,  -2.784643  ,  13.663697  , ...,  24.555475  ,
         11.00234   ,   6.414037  ],
       [ -4.793739  , -17.676325  , -24.432667  , ...,  11.525996  ,
        -35.70123   ,   1.1551397 ]], dtype=float32)>

In [0]:
class LSHasher:
    def __init__(self,hash_size = 5, input_dim = 512):
        self.hash_size = hash_size
        self.input_dim = input_dim
        self.projections = tf.random.normal((self.input_dim, self.hash_size))
    def get_hash(self,inp):
        res = inp@self.projections
        res = tf.cast(res > 0,tf.int32).numpy()
        df = pd.DataFrame(res)
        df['hash'] = df.apply(lambda row: str(row[0])+str(row[1])+str(row[2])+str(row[3])+str(row[4]),1)
        return df

In [45]:
lsh = LSHasher(5,512)
fake_emb = tf.random.normal([1000000,512])
hash1 = lsh.get_hash(fake_emb)
hash1

,0,1,2,3,4,hash
0,0,0,0,0,0,00000
1,0,1,1,0,1,01101
2,1,0,0,0,1,10001
3,0,1,1,1,1,01111
4,1,0,0,1,1,10011
...,...,...,...,...,...,...
999995,1,1,1,0,0,11100
999996,0,1,0,1,1,01011
999997,1,0,1,0,0,10100
999998,1,0,0,1,0,10010


In [43]:
pd.DataFrame(np.concatenate([hash1.numpy(),np.random.randn(1000000,1)],1)).groupby([0,1,2,3,4]).count()

5
0   1   2   3   4         
0.0 0.0 0.0 0.0 0.0  25729
                1.0  31748
            1.0 0.0  27944
                1.0  33200
        1.0 0.0 0.0  32629
                1.0  33532
            1.0 0.0  29514
                1.0  29203
    1.0 0.0 0.0 0.0  30569
                1.0  33161
            1.0 0.0  32047
                1.0  33154
        1.0 0.0 0.0  36161
                1.0  32306
            1.0 0.0  32069
                1.0  26546
1.0 0.0 0.0 0.0 0.0  26561
                1.0  31743
            1.0 0.0  32551
                1.0  36413
        1.0 0.0 0.0  32763
                1.0  32146
            1.0 0.0  32807
                1.0  30633
    1.0 0.0 0.0 0.0  29068
                1.0  30131
            1.0 0.0  33608
                1.0  32529
        1.0 0.0 0.0  33668
                1.0  28504
            1.0 0.0  31635
                1.0  25728

In [13]:
from lshash import LSHash
 
k = 10 # hash size
L = 5  # number of tables
d = 512 # Dimension of Feature vector
lsh = LSHash(hash_size=k, input_dim=d, num_hashtables=L)
# LSH on all the images
for img_path, vec in tqdm_notebook(feature_dict.items()):
    lsh.index(vec.flatten(), extra_data=img_path)

ModuleNotFoundError: ignored

In [14]:
!pip install lhash

ERROR: Could not find a version that satisfies the requirement lhash (from versions: none)
ERROR: No matching distribution found for lhash
